In [17]:
# [1] --- Diagnóstico y imports ---
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Mostrar directorio de trabajo y listar archivos en data/raw para diagnosticar rutas
cwd = Path('.').resolve()
print('Working dir:', cwd)
raw_dir = cwd / '..' / 'data' / 'raw'
raw_dir = raw_dir.resolve()
print('Looking for data in:', raw_dir)
if raw_dir.exists():
    files = sorted([p.name for p in raw_dir.iterdir()])
    print('Files in data/raw:')
    for f in files:
        print(' -', f)
else:
    print('Directory data/raw not found. Check your workspace layout.')


Working dir: C:\Users\Dagok\OneDrive\Documents\GitHub\lomarosa-data\notebooks
Looking for data in: C:\Users\Dagok\OneDrive\Documents\GitHub\lomarosa-data\data\raw
Files in data/raw:
 - .gitkeep
 - INVENTARIO_LOMAROSA.xlsx
 - consolidado -1.xlsx


In [18]:
# [2] --- Lectura robusta del Excel ---
from pathlib import Path
raw_dir = Path('.') / '..' / 'data' / 'raw'
raw_dir = raw_dir.resolve()
# Intentar encontrar un archivo xlsx automáticamente (coincidencia por nombre) 
candidates = []
if raw_dir.exists():
    for p in raw_dir.iterdir():
        if p.is_file() and p.suffix.lower() in ('.xlsx', '.xls'):
            candidates.append(p)

if not candidates:
    raise FileNotFoundError(f'No se encontraron archivos Excel en {raw_dir}.')

# Priorizar archivos que contienen 'consolid' o 'invent' en el nombre, si existen
def score(p):
    name = p.name.lower()
    score = 0
    if 'consolid' in name: score += 10
    if 'invent' in name or 'inventario' in name: score += 8
    if 'planta' in name: score += 5
    return score

candidates = sorted(candidates, key=lambda p: score(p), reverse=True)
chosen = candidates[0]
print('Using file:', chosen)
# Leer hoja por defecto; si hay problemas con encabezados, se puede ajustar header=...
df = pd.read_excel(chosen)
print('Read dataframe with shape:', df.shape)
print('Columns:', list(df.columns))


Using file: C:\Users\Dagok\OneDrive\Documents\GitHub\lomarosa-data\data\raw\consolidado -1.xlsx
Read dataframe with shape: (33, 5)
Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Read dataframe with shape: (33, 5)
Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']


In [19]:
# [7] --- Comparativo Inventario vs Promedio Simple ---
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta

# 1. Leer datos históricos (solo las columnas necesarias)
consolidado_path = Path('.').resolve().parent / 'data' / 'raw' / 'consolidado -1.xlsx'
df_hist = pd.read_excel(
    consolidado_path, 
    sheet_name='Sheet1',
    usecols=['Local', 'Doc', 'Kg totales2', 'Cod']
)

# Mostrar información inicial
print("=== Información inicial de datos históricos ===")
print(f"Total de registros: {len(df_hist)}")
print("\nTipos de documentos:")
print(df_hist['Doc'].value_counts())
print("\nLocales únicos:")
print(df_hist['Local'].unique())

# Filtrar solo ventas de PLANTA GALAN
df_filtrado = df_hist[
    (df_hist['Doc'].astype(str).str.strip().str.upper() == 'VENTA') &
    (df_hist['Local'].str.contains('PLANTA GALAN', case=False, na=False))
].copy()

print("\n=== Datos después del filtrado ===")
print(f"Registros de ventas en Planta Galán: {len(df_filtrado)}")

# Normalizar códigos
df_filtrado['Cod'] = df_filtrado['Cod'].astype(str).str.strip().str.upper()

# Calcular promedios semanales por producto
promedios = df_filtrado.groupby('Cod').agg(
    Total_Kg=('Kg totales2', 'sum'),
    Num_Ventas=('Kg totales2', 'count')
).reset_index()

# Calcular promedio semanal (asumiendo que los datos son de todo el período)
num_semanas = len(df_filtrado['Cod'].unique())  # Usar como aproximación
promedios['Promedio_Semanal'] = promedios['Total_Kg'] / num_semanas

print("\n=== Estadísticas de ventas ===")
print(f"Productos únicos con ventas: {len(promedios)}")
print("\nEjemplo de promedios (top 5 por volumen):")
print(promedios.nlargest(5, 'Total_Kg')[['Cod', 'Total_Kg', 'Promedio_Semanal']])

# 2. Leer inventario actual
inventario_path = Path('.').resolve().parent / 'data' / 'raw' / 'INVENTARIO_LOMAROSA.xlsx'
df_inv = pd.read_excel(
    inventario_path, 
    sheet_name='CONSOLIDADO',
    skiprows=9
)

print("\n=== Columnas en el archivo de inventario ===")
print(df_inv.columns.tolist())

# Verificar las primeras filas del inventario
print("\n=== Primeras filas del inventario ===")
print(df_inv.head())

# Procesamiento del inventario
# 1. Convertir 'Total' a numérico y limpiar datos
df_inv['Total'] = pd.to_numeric(df_inv['Total'], errors='coerce')
df_inv = df_inv.dropna(subset=['Total'])  # Eliminar filas donde Total es NaN

# Verificar si existe la columna correcta para el código
codigo_columns = [col for col in df_inv.columns if 'cod' in col.lower()]
if codigo_columns:
    print("\n=== Columnas que contienen 'cod' encontradas ===")
    print(codigo_columns)
    codigo_col = codigo_columns[0]  # Usar la primera columna encontrada
else:
    raise KeyError("No se encontró ninguna columna que contenga 'cod' en el DataFrame de inventario")

# Normalizar códigos usando el nombre encontrado
df_inv[codigo_col] = df_inv[codigo_col].astype(str).str.strip().str.upper()

# 2. Filtrar por stock mayor a cero
print("\n=== Datos antes del filtrado ===")
print(f"Número total de productos: {len(df_inv)}")
print(f"Productos con Total = 0: {len(df_inv[df_inv['Total'] == 0])}")
print(f"Productos con Total < 0: {len(df_inv[df_inv['Total'] < 0])}")

df_inv = df_inv[df_inv['Total'] > 0].copy()

print("\n=== Datos después del filtrado (solo Total > 0) ===")
print(f"Número de productos: {len(df_inv)}")
print("\nRango de valores en Total:")
print(f"Mínimo: {df_inv['Total'].min():.2f}")
print(f"Máximo: {df_inv['Total'].max():.2f}")

# 3. Hacer la comparación
comparacion = pd.merge(
    promedios[['Cod', 'Promedio_Semanal', 'Total_Kg', 'Num_Ventas']], 
    df_inv[[codigo_col, 'Productos', 'Total']],
    left_on='Cod',
    right_on=codigo_col,
    how='inner'
)

# Limpiar y calcular diferencias
comparacion = comparacion.rename(columns={
    'Total': 'Stock_Actual',
    'Productos': 'Producto'
})
comparacion = comparacion.drop(codigo_col, axis=1)
comparacion['Diferencia'] = comparacion['Stock_Actual'] - comparacion['Promedio_Semanal']
comparacion['Estado'] = comparacion['Diferencia'].apply(
    lambda x: 'Exceso' if x > 0 else 'Faltante' if x < 0 else 'OK'
)

print("\n=== Resultados de la comparación ===")
print(f"Productos con datos históricos y en inventario: {len(comparacion)}")
print(f"Productos solo en inventario: {len(df_inv) - len(comparacion)}")

# Mostrar ejemplos de la comparación
print("\n=== Ejemplos de comparación ===")
print("Top 5 productos con mayor diferencia (positiva o negativa):")
comparacion['Diferencia_Abs'] = comparacion['Diferencia'].abs()
print(comparacion.nlargest(5, 'Diferencia_Abs')[
    ['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia', 'Estado']
].to_string())

# Guardar resultados para uso en otras celdas
comparison = comparacion  # Para mantener compatibilidad con otras celdas
df_inv_sin_total = df_inv[df_inv['Total'] > 0].copy()  # Para mantener compatibilidad con otras celdas

=== Información inicial de datos históricos ===
Total de registros: 200197

Tipos de documentos:
Doc
Venta             124078
TRANSFORMACION     46037
SALIDA             12917
DESPOSTE            6083
TRASLADO            3575
ENTRADA             3025
Devolucion          1846
INVENTARIO          1700
MERMA                483
Compras              314
Nota Credito         127
Nota                  12
Name: count, dtype: int64

Locales únicos:
['PLANTA GALAN' 'LOM' nan 'M.GALAN' 'Otros/ oficina' 'Otros']

=== Datos después del filtrado ===
Registros de ventas en Planta Galán: 12067

=== Estadísticas de ventas ===
Productos únicos con ventas: 80

Ejemplo de promedios (top 5 por volumen):
    Cod    Total_Kg  Promedio_Semanal
32  316  122509.180       1531.364750
11  256   36052.025        450.650313
8   251   31570.170        394.627125
31  315   29371.350        367.141875
33  317   28658.590        358.232375

=== Estadísticas de ventas ===
Productos únicos con ventas: 80

Ejemplo de prom

In [20]:
# [8] --- Visualización HTML: Pie Chart de Disponibilidad ---
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML

# Filtrar productos por debajo del promedio y calcular KPIs
# Consideramos que un producto está bajo promedio si su stock actual es menor que su promedio semanal
productos_faltantes = comparison[comparison['Stock_Actual'] < comparison['Promedio_Semanal']].copy()
num_faltantes = len(productos_faltantes)

# Preparar datos para el pie chart
total_productos = len(comparison)  # Cambio: usar len(comparison) en lugar de len(df_inv_sin_total)
disponibles = total_productos - num_faltantes

# Crear figura para el pie chart
fig_pie = go.Figure()

# Pie chart
fig_pie.add_trace(
    go.Pie(
        labels=["Stock Bajo Promedio", "Stock Adecuado"],
        values=[num_faltantes, disponibles],
        hole=0.5,
        marker_colors=['#ff4444', '#4CAF50'],
        textinfo='value+percent',
        textposition='inside',
        showlegend=True
    )
)

# Actualizar el layout
fig_pie.update_layout(
    title_text="<b>Estado del Stock</b>",
    title_x=0,
    title_font_size=24,
    title_font_color='#4254b5',
    showlegend=True,
    height=500,
    width=600,
    margin=dict(t=100, b=0, l=0, r=0)
)

fig_pie.show()

# Mostrar estadísticas
print("\n=== Estadísticas de Stock ===")
print(f"Total productos comparados: {total_productos}")
print(f"Productos con stock bajo promedio: {num_faltantes}")
print(f"Productos con stock adecuado: {disponibles}")

# Verificar si hay productos con stock cero
stock_cero = comparison[comparison['Stock_Actual'] == 0]
if len(stock_cero) > 0:
    print("\n=== ATENCIÓN: Productos con Stock Cero ===")
    print(stock_cero[['Producto', 'Stock_Actual', 'Promedio_Semanal']])


=== Estadísticas de Stock ===
Total productos comparados: 0
Productos con stock bajo promedio: 0
Productos con stock adecuado: 0


In [14]:
# Diagnóstico de la comparación
print("=== Diagnóstico de datos ===")
print(f"Total de productos en comparison: {len(comparison)}")
print("\nEstadísticas de la diferencia:")
print(comparison['Diferencia'].describe())

# Crear gráfico de barras para todos los productos
import plotly.graph_objects as go

# Ordenar productos por la diferencia absoluta
comparison['Diferencia_Abs'] = comparison['Diferencia'].abs()
productos_ordenados = comparison.sort_values('Diferencia_Abs', ascending=True)

# Crear figura
fig = go.Figure()

# Agregar barras para el promedio semanal
fig.add_trace(
    go.Bar(
        y=productos_ordenados['Producto'],
        x=productos_ordenados['Promedio_Semanal'],
        name='Promedio Semanal',
        orientation='h',
        marker_color='#4CAF50'
    )
)

# Agregar barras para el stock actual
fig.add_trace(
    go.Bar(
        y=productos_ordenados['Producto'],
        x=productos_ordenados['Stock_Actual'],
        name='Stock Actual',
        orientation='h',
        marker_color='#2196F3'
    )
)

# Actualizar el layout
fig.update_layout(
    title_text="<b>Comparación de Stock Actual vs Promedio Semanal</b>",
    barmode='group',
    height=max(600, len(productos_ordenados) * 25),  # Altura dinámica según número de productos
    width=1000,
    yaxis={'categoryorder':'total ascending'},
    xaxis_title="Kilogramos",
    showlegend=True,
    margin=dict(l=250)  # Margen izquierdo para nombres de productos
)

fig.show()

# Mostrar tabla con todos los productos y sus métricas
print("\n=== Detalle de todos los productos ===")
tabla_detalle = comparison[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia', 'Estado']].copy()
tabla_detalle = tabla_detalle.sort_values('Diferencia')
print(tabla_detalle.to_string())

=== Diagnóstico de datos ===
Total de productos en comparison: 0

Estadísticas de la diferencia:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Diferencia, dtype: float64



=== Detalle de todos los productos ===
Empty DataFrame
Columns: [Producto, Stock_Actual, Promedio_Semanal, Diferencia, Estado]
Index: []


In [6]:
# [9] --- Indicadores KPI ---
import plotly.graph_objects as go

# Filtrar el DataFrame para excluir la fila de total (última fila)
df_inv_sin_total = df_inv.iloc[:-1].copy()

# Calcular KPIs
total_productos = len(df_inv_sin_total)
productos_disponibles = total_productos - num_faltantes
stock_total_kg = df_inv_sin_total['Total'].sum()

# Crear figura para los indicadores
fig_kpi = go.Figure()

# 1. Total Productos
fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=total_productos,
    title={"text": "Total Productos"},
    domain={'row': 0, 'column': 0},
    number={"font": {"size": 50}}
))

# 2. Productos con Stock Adecuado
fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=productos_disponibles,
    title={"text": "Stock Adecuado"},
    domain={'row': 0, 'column': 1},
    number={"font": {"size": 50, "color": "green"}}
))

# 3. Stock Total
fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=stock_total_kg,
    title={"text": "Stock Total (Kg)"},
    domain={'row': 0, 'column': 2},
    number={"font": {"size": 50, "color": "darkblue"},
            "valueformat": ",.1f"}
))

# 4. Productos bajo promedio
fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=num_faltantes,
    title={"text": "Bajo Promedio"},
    domain={'row': 0, 'column': 3},
    number={"font": {"size": 50, "color": "red"}}
))

# Actualizar el layout para mostrar los indicadores en línea
fig_kpi.update_layout(
    grid={'rows': 1, 'columns': 4},
    height=250,
    width=1200,
    title=dict(
        text="<b>Indicadores de Inventario</b>",
        x=0,
        y=0.95,
        xanchor='left',
        yanchor='top',
        font=dict(
            size=24,
            color='#4254b5'
        )
    ),
    margin=dict(t=80, b=20, l=20, r=20)
)

fig_kpi.show()

In [11]:
# [8.1] --- Visualización: Top 5 Productos Bajo Promedio ---
import plotly.graph_objects as go

# Verificar los datos antes de crear la visualización
print("=== Verificación de datos ===")
print("\nColumnas en productos_faltantes:")
print(productos_faltantes.columns.tolist())
print("\nNúmero de productos faltantes:", len(productos_faltantes))
if len(productos_faltantes) > 0:
    print("\nPrimeros registros de productos_faltantes:")
    print(productos_faltantes[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia']].head().to_string())

# Ordenar por el valor absoluto de la diferencia y tomar los top 5
top5_faltantes = productos_faltantes.copy()
top5_faltantes['Diferencia_Abs'] = abs(top5_faltantes['Diferencia'])
top5_faltantes = top5_faltantes.nlargest(5, 'Diferencia_Abs')

print("\nTop 5 productos con mayor diferencia:")
print(top5_faltantes[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia', 'Diferencia_Abs']].to_string())

# Crear figura para el bar chart
fig_bar_bajo = go.Figure()

# Agregar barras para el promedio semanal (rojas)
fig_bar_bajo.add_trace(
    go.Bar(
        y=top5_faltantes['Producto'],
        x=top5_faltantes['Promedio_Semanal'],
        orientation='h',
        name='Promedio Semanal',
        marker_color='#ff4444',
        text=top5_faltantes['Promedio_Semanal'].round(1),
        textposition='outside',
    )
)

# Agregar barras para el stock actual (naranjas)
fig_bar_bajo.add_trace(
    go.Bar(
        y=top5_faltantes['Producto'],
        x=top5_faltantes['Stock_Actual'],
        orientation='h',
        name='Stock Actual',
        marker_color='#FF9800',
        text=top5_faltantes['Stock_Actual'].round(1),
        textposition='outside',
    )
)

# Agregar anotaciones para mostrar el gap
for idx, row in enumerate(top5_faltantes.itertuples()):
    gap = abs(row.Diferencia)
    fig_bar_bajo.add_annotation(
        x=(row.Stock_Actual + row.Promedio_Semanal)/2,
        y=row.Producto,
        text=f'Gap: {gap:.1f} kg',
        showarrow=False,
        font=dict(size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.8)'
    )

# Actualizar el layout
fig_bar_bajo.update_layout(
    title_text="<b>Top 5 Productos Bajo Promedio</b>",
    title_x=0,
    title_font_size=24,
    title_font_color='#4254b5',
    showlegend=True,
    height=500,
    width=800,
    margin=dict(t=100, b=0, l=0, r=0),
    barmode='overlay',
    xaxis_title="Kilogramos",
    yaxis_title=""
)

fig_bar_bajo.show()

=== Verificación de datos ===

Columnas en productos_faltantes:
['Cod', 'Promedio_Semanal', 'Total_Kg', 'Num_Ventas', 'Producto', 'Stock_Actual', 'Diferencia', 'Estado', 'Diferencia_Abs']

Número de productos faltantes: 0

Top 5 productos con mayor diferencia:
Empty DataFrame
Columns: [Producto, Stock_Actual, Promedio_Semanal, Diferencia, Diferencia_Abs]
Index: []


In [ ]:
# [8.2] --- Visualización: Top 5 Productos con Más Stock ---
import plotly.graph_objects as go

# Verificar los datos del DataFrame comparison
print("=== Verificación de datos ===")
print("\nColumnas en comparison:")
print(comparison.columns.tolist())
print("\nNúmero total de productos:", len(comparison))
if len(comparison) > 0:
    print("\nPrimeros registros de comparison:")
    print(comparison[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia']].head().to_string())

# Ordenar productos por stock actual y tomar los top 5
top5_mas_stock = comparison.nlargest(5, 'Stock_Actual')

print("\nTop 5 productos con más stock:")
print(top5_mas_stock[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia']].to_string())

# Crear figura para el bar chart
fig_bar_alto = go.Figure()

# Agregar barras para el promedio semanal (verde)
fig_bar_alto.add_trace(
    go.Bar(
        y=top5_mas_stock['Producto'],
        x=top5_mas_stock['Promedio_Semanal'],
        orientation='h',
        name='Promedio Semanal',
        marker_color='#4CAF50',
        text=top5_mas_stock['Promedio_Semanal'].round(1),
        textposition='outside',
    )
)

# Agregar barras para el stock actual (azul)
fig_bar_alto.add_trace(
    go.Bar(
        y=top5_mas_stock['Producto'],
        x=top5_mas_stock['Stock_Actual'],
        orientation='h',
        name='Stock Actual',
        marker_color='#2196F3',
        text=top5_mas_stock['Stock_Actual'].round(1),
        textposition='outside',
    )
)

# Agregar anotaciones para mostrar el gap
for idx, row in enumerate(top5_mas_stock.itertuples()):
    gap = abs(row.Diferencia)
    fig_bar_alto.add_annotation(
        x=(row.Stock_Actual + row.Promedio_Semanal)/2,
        y=row.Producto,
        text=f'Gap: {gap:.1f} kg',
        showarrow=False,
        font=dict(size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.8)'
    )

# Actualizar el layout
fig_bar_alto.update_layout(
    title_text="<b>Top 5 Productos con Más Stock</b>",
    title_x=0,
    title_font_size=24,
    title_font_color='#4254b5',
    showlegend=True,
    height=500,
    width=800,
    margin=dict(t=100, b=0, l=0, r=0),
    barmode='overlay',
    xaxis_title="Kilogramos",
    yaxis_title=""
)

fig_bar_alto.show()

# Verificar los datos para la alerta
print("\n=== Datos para la alerta ===")
print(f"Número de productos faltantes: {num_faltantes}")

# Crear alerta visual estilo banner
alerta_html = f"""
<div style="background-color: #ffe6e6; padding: 20px; border-radius: 5px; margin: 20px 0;">
    <h2 style="color: #333; margin: 0;">
        <span style="color: #ff4444; margin-right: 10px;">🔔</span>
        Alerta Crítica
    </h2>
    <p style="font-size: 16px; margin: 10px 0 0 0;">
        Hay <strong>{num_faltantes} producto(s)</strong> sin stock que requieren atención inmediata.
    </p>
</div>
"""
display(HTML(alerta_html))

# Verificar datos para la tabla
print("\n=== Datos para la tabla de faltantes ===")
print("Número de productos faltantes:", len(productos_faltantes))

# Tabla HTML de productos faltantes y cuánto falta para el promedio
tabla_faltantes = productos_faltantes[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Diferencia']].copy()
tabla_faltantes['Faltante_para_Promedio'] = tabla_faltantes['Diferencia'].abs().round(2)
tabla_faltantes = tabla_faltantes.rename(columns={
    'Producto': 'Producto',
    'Stock_Actual': 'Stock Actual',
    'Promedio_Semanal': 'Promedio Semanal (Kg)',
    'Faltante_para_Promedio': 'Faltante para Promedio (Kg)'
})

if len(tabla_faltantes) > 0:
    html_table = tabla_faltantes[['Producto', 'Stock Actual', 'Promedio Semanal (Kg)', 'Faltante para Promedio (Kg)']].to_html(index=False)
    display(HTML("<h3>Productos por debajo del promedio semanal</h3>" + html_table))
else:
    print("No hay productos por debajo del promedio semanal")

Producto,Stock Actual,Promedio Semanal (Kg),Faltante para Promedio (Kg)


In [10]:
# [8.3] --- Tabla Completa de Productos Bajo Promedio ---
from IPython.display import display, HTML
import pandas as pd

# Ordenar productos por la cantidad faltante (de mayor a menor)
tabla_completa = productos_faltantes[['Producto', 'Stock_Actual', 'Promedio_Semanal_Kg', 'Diferencia']].copy()
tabla_completa['Faltante_para_Promedio'] = tabla_completa['Diferencia'].abs().round(2)
tabla_completa = tabla_completa.sort_values('Faltante_para_Promedio', ascending=False)

# Renombrar columnas y dar formato
tabla_formateada = tabla_completa.rename(columns={
    'Producto': 'Producto',
    'Stock_Actual': 'Stock Actual (Kg)',
    'Promedio_Semanal_Kg': 'Promedio Semanal (Kg)',
    'Faltante_para_Promedio': 'Faltante para Promedio (Kg)'
}).drop('Diferencia', axis=1)

# Aplicar formato numérico a las columnas
for col in ['Stock Actual (Kg)', 'Promedio Semanal (Kg)', 'Faltante para Promedio (Kg)']:
    tabla_formateada[col] = tabla_formateada[col].round(2)

# Crear estilo HTML para la tabla
tabla_style = """
<style>
    table {
        border-collapse: collapse;
        margin: 25px 0;
        font-size: 14px;
        font-family: sans-serif;
        min-width: 400px;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.15);
    }
    table thead tr {
        background-color: #4254b5;
        color: #ffffff;
        text-align: left;
    }
    table th,
    table td {
        padding: 12px 15px;
    }
    table tbody tr {
        border-bottom: 1px solid #dddddd;
    }
    table tbody tr:nth-of-type(even) {
        background-color: #f3f3f3;
    }
    table tbody tr:last-of-type {
        border-bottom: 2px solid #4254b5;
    }
</style>
"""

# Convertir DataFrame a HTML con estilos
html_table = tabla_formateada.to_html(index=False, classes=['styled-table'])
html_content = f"""
{tabla_style}
<div style="margin: 20px 0;">
    <h2 style="color: #4254b5; margin-bottom: 10px;">Análisis Detallado de Productos Bajo Promedio</h2>
    <p style="color: #666; margin-bottom: 20px;">
        Total de productos bajo promedio: <strong>{len(tabla_formateada)}</strong><br>
        Ordenados por cantidad faltante (de mayor a menor necesidad)
    </p>
    {html_table}
</div>
"""

display(HTML(html_content))

KeyError: "['Promedio_Semanal_Kg'] not in index"

In [ ]:
# [10] --- Lista de Productos y Códigos ---
print("=== Lista de Productos y sus Códigos ===")

# Asumiendo que estamos usando el DataFrame comparison que tiene la información más completa
productos_codigos = comparison[['Cod', 'Producto']].sort_values('Producto')

# Crear una tabla formateada
from IPython.display import display, HTML

# Estilo para la tabla HTML
tabla_style = """
<style>
    .productos-table {
        border-collapse: collapse;
        margin: 25px 0;
        font-size: 14px;
        font-family: sans-serif;
        min-width: 400px;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.15);
    }
    .productos-table thead tr {
        background-color: #4254b5;
        color: #ffffff;
        text-align: left;
    }
    .productos-table th,
    .productos-table td {
        padding: 12px 15px;
    }
    .productos-table tbody tr {
        border-bottom: 1px solid #dddddd;
    }
    .productos-table tbody tr:nth-of-type(even) {
        background-color: #f3f3f3;
    }
    .productos-table tbody tr:last-of-type {
        border-bottom: 2px solid #4254b5;
    }
</style>
"""

# Convertir DataFrame a HTML con estilos
html_table = productos_codigos.to_html(index=False, 
                                     classes=['productos-table'],
                                     table_id='tabla-productos')

html_content = f"""
{tabla_style}
<div style="margin: 20px 0;">
    <h2 style="color: #4254b5; margin-bottom: 10px;">Catálogo de Productos</h2>
    <p style="color: #666; margin-bottom: 20px;">
        Total de productos: <strong>{len(productos_codigos)}</strong>
    </p>
    {html_table}
</div>
"""

display(HTML(html_content))

# También mostrar en formato texto plano para facilitar la copia
print("\nListado en formato texto:")
print(productos_codigos.to_string(index=False))